# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [95]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))

adult_dt.head(10)


age          workclass  fnlwgt   education  education-num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   
5   37            Private  284582     Masters             14   
6   49            Private  160187         9th              5   
7   52   Self-emp-not-inc  209642     HS-grad              9   
8   31            Private   45781     Masters             14   
9   42            Private  159449   Bachelors             13   

           marital-status          occupation    relationship    race  \
0           Never-married        Adm-clerical   Not-in-family   White   
1      Married-civ-spouse     Exec-managerial         Husband   White   
2                Divorced   Handlers-cleaners   Not-in-family   White   
3      Married-civ-spouse   Handlers-cleaners         Husband   Black   
4      Married-civ-spouse      Prof-specialty            Wife   Black   
5      Married-civ-spouse     Exec-managerial            Wife   White   
6   Married-spouse-absent       Other-service   Not-in-family   Black   
7      Married-civ-spouse     Exec-managerial         Husband   White   
8           Never-married      Prof-specialty   Not-in-family   White   
9      Married-civ-spouse     Exec-managerial         Husband   White   

       sex  capital-gain  capital-loss  hours-per-week  native-country  income  
0     Male          2174             0              40   United-States       0  
1     Male             0             0              13   United-States       0  
2     Male             0             0              40   United-States       0  
3     Male             0             0              40   United-States       0  
4   Female             0             0              40            Cuba       0  
5   Female             0             0              40   United-States       0  
6   Female             0             0              16         Jamaica       0  
7     Male             0             0              45   United-States       1  
8   Female         14084             0              50   United-States       1  
9     Male          5178             0              40   United-States       1

# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [96]:
from sklearn.model_selection import train_test_split

X = adult_dt.drop(['income'], axis=1)
# X.head()
Y = adult_dt['income']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 

Random state is the seed value that is supplied to the RNG algorithm.

+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?
 
 It allows for the generation of the same random values across multiple runs.

## Answers

#### What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 

**Random state is the seed value that is supplied to the RNG algorithm.**

#### Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?
 
 **It allows for the generation of the same random values across multiple runs.**

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [97]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, PowerTransformer, RobustScaler, OneHotEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import GaussianNB

num_cols = [ 'age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
cat_cols = ['workclass','education','marital-status','occupation', 'relationship', 'race', 'sex', 'native-country']
num_transformer = Pipeline(
    steps = [
        ('knn_imputer', KNNImputer(n_neighbors=7, weights='distance')),
        ('robustScaler', RobustScaler()),
    ]
)
cat_transformer = Pipeline(
    steps = [
        ('simple_imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown="infrequent_if_exist", drop="if_binary"))
        ]
)

transformer = ColumnTransformer(
    transformers=[
        ('num_transforms', num_transformer, num_cols),
        ("cat_transforms", cat_transformer, cat_cols) 
    ], remainder='drop'
)

###     use infrequent_if_exists because Holand-Netherlands is apparently not a thing in one of the sets. So transform it
###     to all zeros and put it at the bottom using infrequent_if_exist. This _will_ send out a warning, to stdout,
###     but such is life.

###    raise ValueError(msg)
###     ValueError: Found unknown categories [' Holand-Netherlands'] in column 7 during transform

transformer

ColumnTransformer(transformers=[('num_transforms',
                                 Pipeline(steps=[('knn_imputer',
                                                  KNNImputer(n_neighbors=7,
                                                             weights='distance')),
                                                 ('robustScaler',
                                                  RobustScaler())]),
                                 ['age', 'fnlwgt', 'education-num',
                                  'capital-gain', 'capital-loss',
                                  'hours-per-week']),
                                ('cat_transforms',
                                 Pipeline(steps=[('simple_imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(drop='if_binary',
                                                                handle_unknown='infrequent_if_exist'))]),
                                 ['workclass', 'education', 'marital-status',
                                  'occupation', 'relationship', 'race', 'sex',
                                  'native-country'])])

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [98]:
from sklearn.ensemble import RandomForestClassifier
pipe = Pipeline(
    [
        ("preprocessing", transformer),
        ('classifier', RandomForestClassifier())
    ]
)
pipe

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num_transforms',
                                                  Pipeline(steps=[('knn_imputer',
                                                                   KNNImputer(n_neighbors=7,
                                                                              weights='distance')),
                                                                  ('robustScaler',
                                                                   RobustScaler())]),
                                                  ['age', 'fnlwgt',
                                                   'education-num',
                                                   'capital-gain',
                                                   'capital-loss',
                                                   'hours-per-week']),
                                                 ('cat_transforms',
                                                  Pipeline(steps=[('simple_imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(drop='if_binary',
                                                                                 handle_unknown='infrequent_if_exist'))]),
                                                  ['workclass', 'education',
                                                   'marital-status',
                                                   'occupation', 'relationship',
                                                   'race', 'sex',
                                                   'native-country'])])),
                ('classifier', RandomForestClassifier())])

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [99]:
from sklearn.model_selection import cross_validate, KFold
pipe.fit(X_train,Y_train)

scoring = ['roc_auc', 'neg_log_loss', 'accuracy', 'balanced_accuracy']
dict_score = cross_validate(pipe, X_train, Y_train, cv=5, scoring= scoring, return_train_score = True)

/home/stumac/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(
/home/stumac/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/sklearn/preprocessing/_encoders.py:242: UserWarning: Found unknown categories in columns [7] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [100]:
df = pd.DataFrame(dict_score).sort_values(by='test_neg_log_loss')
df

fit_time  score_time  test_roc_auc  train_roc_auc  test_neg_log_loss  \
4  5.318187    0.152203      0.902425            1.0          -0.406285   
3  5.220157    0.140294      0.907259            1.0          -0.377153   
2  5.169781    0.123925      0.902412            1.0          -0.375033   
1  4.967463    0.149261      0.903027            1.0          -0.372432   
0  5.076757    0.150482      0.905104            1.0          -0.362312   

   train_neg_log_loss  test_accuracy  train_accuracy  test_balanced_accuracy  \
4           -0.081590       0.856955        1.000000                0.779130   
3           -0.082240       0.860026        1.000000                0.782388   
2           -0.081413       0.853225        1.000000                0.776361   
1           -0.081332       0.850406        1.000000                0.773110   
0           -0.081170       0.848212        0.999945                0.771355   

   train_balanced_accuracy  
4                 1.000000  
3                 1.000000  
2                 1.000000  
1                 1.000000  
0                 0.999964

Calculate the mean of each metric. 

In [101]:
df.mean()

fit_time                   5.150469
score_time                 0.143233
test_roc_auc               0.904046
train_roc_auc              1.000000
test_neg_log_loss         -0.378643
train_neg_log_loss        -0.081549
test_accuracy              0.853765
train_accuracy             0.999989
test_balanced_accuracy     0.776469
train_balanced_accuracy    0.999993
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [102]:
from sklearn.metrics import accuracy_score, log_loss, roc_auc_score, balanced_accuracy_score
Y_pred_test = pipe.predict(X_test)
Y_proba_test = pipe.predict_proba(X_test)

test_res = {
    'accuracy_score': accuracy_score(Y_test, Y_pred_test),
    'balanced_accuracy_score': balanced_accuracy_score(Y_test, Y_pred_test),
    'neg_log_loss_score': -log_loss(Y_test, Y_proba_test),
    'roc_auc_score': roc_auc_score(Y_test, Y_proba_test[:,1]),
}

test_res

{'accuracy_score': 0.8547446002661481,
 'balanced_accuracy_score': 0.7756399242472989,
 'neg_log_loss_score': -0.3600082112820292,
 'roc_auc_score': 0.9013173248653545}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

moving the income into a binary case (over or under 50 equalling 0 or 1) allows us to run the classifiers against the model, which make probability estimates possible. Given that the data provides 50k as the water mark between a "high earner" and a "low earning" in this binary classification, it probably means that someone in the business side of things considers that limit to
be important.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.